In [2]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ['https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive']

#개인에 따라 수정 필요 - 다운로드 받았던 키 값 경로 
json_key_path = "enviroments/abnormal-situation-leaderboard-3ca42d06719e.json"	

credential = ServiceAccountCredentials.from_json_keyfile_name(json_key_path, scope)
gc = gspread.authorize(credential)


#개인에 따라 수정 필요 - 스프레드시트 url 가져오기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1MY0okfx4niDAH1SovZ8Wh1K9QosL2Cww09ORtPrC-yQ/edit?gid=0#gid=0"

doc = gc.open_by_url(spreadsheet_url)

#개인에 따라 수정 필요 - 시트 선택하기 (시트명을 그대로 입력해주면 된다.)
sheet = doc.worksheet("시트1")

#데이터 프레임 생성하기
df = pd.DataFrame(sheet.get_all_values())

#불러온 데이터 프레임 정리
df.rename(columns=df.iloc[0], inplace = True)
df.drop(df.index[0], inplace=True)

df.head()

,Model,Model name,Model link,Datasets Used,Evaluation Categories,Average F1,Hub License,TASK,#Params (B),군중밀집,배회,쓰러짐,담넘기,쓰레기 투척,화재,터널내이상상황,폭력
1,"<a target=""_blank"" href=""https://huggingface.c...",PIA-SPACE-LAB/20241014_FT_CLIP4Clip_KTT,https://huggingface.co/PIA-SPACE-LAB/20241014_...,"KTT, GJ","Disaster, Fire",0.85,Apache-2.0,🔶Video Retrieval,2.5,0.8,0.7,0.9,FALSE,0.65,0.95,0.75,FALSE
2,"<a target=""_blank"" href=""https://huggingface.c...",PIA-SPACE-LAB/20241014_FT_CLIP4Clip_KTT,https://huggingface.co/PIA-SPACE-LAB/20241014_...,Songpa,"Disaster, Violence",0.78,MIT,🔶Video Retrieval,1.8,0.65,FALSE,0.78,0.6,0.55,FALSE,0.7,0.8
3,"<a target=""_blank"" href=""https://huggingface.c...",PIA-SPACE-LAB/20241014_FT_CLIP4Clip_KTT,https://huggingface.co/PIA-SPACE-LAB/20241014_...,"KTT, Songpa",Fire,0.92,Apache-2.0,🔶Video Retrieval,3.2,FALSE,0.85,0.88,0.75,FALSE,0.98,0.88,0.7
4,"<a target=""_blank"" href=""https://huggingface.c...",PIA-SPACE-LAB/20241014_FT_CLIP4Clip_KTT,https://huggingface.co/PIA-SPACE-LAB/20241014_...,"KTT, Songpa",Fire,0.92,Apache-2.0,,,,,,,,,,
